# Processing webtables

In [1]:
%%capture --no-display
import logging as log
log.getLogger().setLevel(log.DEBUG)

import takco
params = {
    'class': 'WebDataCommons',
    'fnames': [
#     f'hdfs://bricks07:9000/user/kruit/wdc_webtables_englishCorpus_2015-07/xaa.jsonl'
        '/bigstore/kruit/webtables/englishCorpus_2015-07_filtered/xaa.jsonl'
    ]
}
tables = takco.TableSet.dataset(
    params=params,
    executor={'class': 'DaskHashBag', 'address': 'tcp://192.168.62.207:8686'}
).tables
tables.client if hasattr(tables, 'client') else None

Client Scheduler: tcp://192.168.62.207:8686 Dashboard: http://192.168.62.207:8687/status,Cluster Workers: 6 Cores: 48 Memory: 1.62 TB


In [2]:
tables = tables.persist()
tables.bag.count().compute() if hasattr(tables, 'bag') else sum(1 for _ in tables)

8924

In [3]:
typer_config = {
    'class':'EntityBloom', 
    'bloomfile':'data/dbpedia_t2ksubset_all_surfaceforms.5pct.bloom', 
    'threshold': 0.5,
}
tables = tables.pipe(takco.TableSet.coltypes, typer_config = typer_config)
takco.preview(tables)

In [4]:
def filter_any_entitycol(ts):
    for t in ts:
        col_cls = t.get("classes", {}).values()
        ent = takco.link.types.SimpleTyper.ENTITY 
        if any(ent in cs for cs in col_cls):
            yield t
        
tables = tables.pipe(filter_any_entitycol).persist()
tables.bag.count().compute() if hasattr(tables, 'bag') else sum(1 for _ in tables)

3171

In [5]:
takco.preview(tables, ntables=20)

In [2]:
%%time
import dask.bag as db
db.read_text('/bigstore/kruit/webtables/englishCorpus_2015-07_filtered/*.jsonl').count().compute()

CPU times: user 2min 41s, sys: 13.9 s, total: 2min 54s
Wall time: 9min 30s


49658538

In [2]:
%%time
import dask.bag as db
import json, itertools
tables = db.read_text('/bigstore/kruit/webtables/englishCorpus_2015-07_filtered/xa*.jsonl').map(json.loads)

sample_n = 1000
sample = tables.map_partitions(lambda xs: list(itertools.islice(xs, sample_n)))
sample = sample.foldby(lambda x: hash(str(x['relation'])), binop = lambda tot, x:x).map(lambda x: x[1])
sample = sample.repartition(tables.npartitions)

import urllib
from collections import Counter

key = lambda x: (urllib.parse.urlparse(x['url']).netloc, next(zip(*x['relation']), None))
counts = dict(sample.foldby(key, initial=0, binop = lambda tot,x: tot+1).compute())
dict(Counter(counts).most_common(10))

CPU times: user 313 ms, sys: 78.6 ms, total: 392 ms
Wall time: 6.61 s


{('www.cappex.com', ('Categories', 'Comments')): 56,
 ('www.reef.org', ('', 'Expert', 'Novice', 'Total')): 24,
 ('www.google.com',
  ('Citing Patent',
   'Filing date',
   'Publication date',
   'Applicant',
   'Title')): 24,
 ('www.bigstockphoto.com', ('', 'px | in | cm', 'Credits')): 23,
 ('www.google.com', ('Date', 'Code', 'Event', 'Description')): 19,
 ('www.dpreview.com', ('Post (hide subjects)', 'Posted by', 'When', '')): 18,
 ('eprints.ucm.es', ('Directive', 'Local Value', 'Master Value')): 18,
 ('www.usms.org', ('#', 'Name', 'Age', 'Club', 'LMSC', 'Time')): 17,
 ('www.appliancefactoryparts.com',
  ('Ref.', 'Image', 'Part No & Description', 'Price', '')): 17,
 ('www.timeanddate.com', ('Location', 'Local time')): 17}

In [3]:
sample.count().compute()

23085

In [5]:
from takco.extract import from_wdc
from takco.util import preview
from takco import TableSet, DaskHashBag, HashBag

# ts = HashBag(sample.compute()).pipe(from_wdc)
ts = DaskHashBag(sample).pipe(from_wdc)
ts = TableSet.cluster(ts, addcontext=['pgTitle'])
ts = TableSet.reshape(ts, restructure=False, compound_splitter_config={'class':'SuffixCompoundSplitter'})
ts = sorted(ts, key=lambda x: -x['numDataRows'])
print(f"Got {len(ts)} tables")
preview(ts, ntables=25)

Got 7007 tables
